## Нормализация "orders_cnt" (количества заказов) по медиане предыдущих 7 дней

In [1]:
import pandas as pd
import arrow 
from tqdm import tqdm
import pickle
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore', category=SettingWithCopyWarning)
# import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
data = pd.read_csv('data/orders.csv')

In [4]:
d0 = data.copy()

Достанем из столбца даты час и день недели

In [5]:
d0['date']=pd.to_datetime(d0['date'])
d0['hour'] = d0.date.dt.hour
d0['weekday'] = d0.date.dt.weekday.astype(int)

In [ ]:
d0 = pd.read_csv('data/d0.csv')

In [7]:
d0['orders_cnt_normalized'] = np.zeros(884856)

In [8]:
normalized_df = pd.DataFrame(columns = ['delivery_area_id', 'date', 'orders_cnt', 'hour', 'weekday', 'orders_cnt_normalized', 'orders_cnt_finally_normalized'])

In [9]:
areas = d0['delivery_area_id'].unique()

In [10]:
import statistics
import statistics
for area in tqdm(areas):
  filtered_df = d0[d0['delivery_area_id'] == area]
  medians = []
  for i in range(7, len(filtered_df), 7):
      # print(filtered_df['orders_cnt'][i-7:i].to_numpy())
      median_new = statistics.median(np.array(filtered_df['orders_cnt'][i-7:i]))
      filtered_df['orders_cnt_normalized'][i-7:i] = median_new
  filtered_df['orders_cnt_finally_normalized'] = filtered_df['orders_cnt'] / filtered_df['orders_cnt_normalized']
  df_append = normalized_df.append(filtered_df, ignore_index=True)
  normalized_df = df_append

100%|██████████| 593/593 [02:02<00:00,  4.85it/s]


In [69]:
# normalized_df.loc[normalized_df.delivery_area_id==0][:20].orders_cnt_finally_normalized.plot()

In [11]:
for i in range(len(normalized_df)):
  if normalized_df['orders_cnt_finally_normalized'][i] == np.inf:
    normalized_df['orders_cnt_finally_normalized'][i] = 0

In [12]:
normalized_df.to_csv('data/normalized_df.csv')